In [1]:
import geopandas as gpd
import shapely
import numpy as np
import matplotlib.pyplot as plt

In [45]:
data = gpd.read_file('/Users/ckruse/Downloads/2022.07.06-earthrise_plastics_exports/Mapping_Plastic_Waste_Challenge.geojson')
label_map = {'Not a waste site': 0, 'Waste site': 1, 'Other (leave note)': 2}
data['label'] = [label_map[l] for l in data['label_class_name']]
data.head()

,id,email,reviews,create_by,created_at,project_id,description,project_name,investigation_id,label_class_name,investigation_name,geometry,label
0,e9eada09-d3ba-4a0c-a03d-a9e30279f093,kale-6d35-4d93@earthrise-2.com,{},kale-6d35-4d93@earthrise-2.com,2022-07-03T07:34:03.143000,03e594d9-c2d7-4fa7-9a3d-40ca700f9dcb,,Mapping Plastic Waste|0|01/01/2022,b820d4f1-8a98-40d7-b304-c7961fa07df5,Not a waste site,Mapping Plastic Waste,POINT (77.88955 9.20878),0
1,ce507c9c-a10f-41b5-81e9-1409c27104e4,kale-6d35-4d93@earthrise-2.com,{},kale-6d35-4d93@earthrise-2.com,2022-07-03T07:34:03.143000,03e594d9-c2d7-4fa7-9a3d-40ca700f9dcb,,Mapping Plastic Waste|0|01/01/2022,b820d4f1-8a98-40d7-b304-c7961fa07df5,Not a waste site,Mapping Plastic Waste,POINT (76.65474 28.87899),0
2,9dea4587-745d-481b-b28e-8947b747acea,kale-6d35-4d93@earthrise-2.com,{},kale-6d35-4d93@earthrise-2.com,2022-07-03T07:34:03.143000,03e594d9-c2d7-4fa7-9a3d-40ca700f9dcb,,Mapping Plastic Waste|0|01/01/2022,b820d4f1-8a98-40d7-b304-c7961fa07df5,Not a waste site,Mapping Plastic Waste,POINT (75.46921 32.12742),0
3,2829e890-5428-4a03-9f9d-693c46e5c750,kale-6d35-4d93@earthrise-2.com,{},kale-6d35-4d93@earthrise-2.com,2022-07-03T07:34:03.143000,03e594d9-c2d7-4fa7-9a3d-40ca700f9dcb,,Mapping Plastic Waste|0|01/01/2022,b820d4f1-8a98-40d7-b304-c7961fa07df5,Not a waste site,Mapping Plastic Waste,POINT (71.49011 24.96031),0
4,e4e8e363-9919-41c3-80e5-3fd2db383eb8,kale-6d35-4d93@earthrise-2.com,{},kale-6d35-4d93@earthrise-2.com,2022-07-03T07:34:03.143000,03e594d9-c2d7-4fa7-9a3d-40ca700f9dcb,,Mapping Plastic Waste|0|01/01/2022,b820d4f1-8a98-40d7-b304-c7961fa07df5,Not a waste site,Mapping Plastic Waste,POINT (77.51580 8.63785),0


In [46]:
print(data['description'][data['label'] == 2].values)
print(len(data))

['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 ' icant see anything' ' i cant see anything' 'i cant see anything'
 'i cant see anything' 'i cant se anything' "i can't see anything"
 "i can't see anything" "i can't see anything" "i can't saw anything"]
2514


In [59]:
positive_labels = gpd.read_file('../../data/site_metadata/compiled_sites_v1.0.geojson')
negative_labels = gpd.GeoDataFrame()
gpd.read_file('../../data/sampling_locations/north_india_east_intersection_confirmed_negatives.geojson')
import pandas as pd
for fn in ['north_india_east_intersection_confirmed_negatives', 'north_india_west_intersection_confirmed_negatives', 'southern_india_intersection_confirmed_negatives']:
    negative_labels = gpd.GeoDataFrame(pd.concat([negative_labels, gpd.read_file(f'../../data/sampling_locations/{fn}.geojson')]))
negative_labels = gpd.GeoDataFrame(negative_labels['geometry'])
negative_labels

,geometry
0,POINT (83.37797 26.75730)
1,POINT (88.22088 23.84751)
2,POINT (83.75260 26.28282)
3,POINT (87.70543 23.88288)
4,POINT (84.03607 27.19509)
...,...
765,POINT (78.32255 9.16695)
766,POINT (76.32775 20.06285)
767,POINT (75.66295 17.47365)
768,POINT (75.47845 15.84887)


In [105]:
point_data = data.copy()
point_data['geometry'] = [geom.centroid for geom in point_data['geometry']]



In [106]:
def query_df(point_df, polygon_df):
    index = polygon_df['geometry'].sindex
    count = 0
    for point in point_df['geometry']:
        if len(index.query(point)) > 0:
            count += 1
    return count

In [95]:
# remove points that overlap from point_data
overlaps = []
for point in point_data['geometry']:
    overlap_count = 0
    for polygon in point_data['geometry']:
        if polygon.overlaps(point):
            overlap_count += 1
    overlaps.append(overlap_count)

KeyboardInterrupt: 

In [147]:
results = {
    'user': [],
    'total labels': [],
    'overlapping labels': [],
    'positive labels': [],
    'negative labels': [],
    'uncertain labels': [],
    'true positive count': [],
    'true negative count': [],
    'confirmed correct count': [],
    'positive accuracy': [],
    'negative accuracy': [],
}
for user in point_data['create_by'].unique():
    user_points = point_data[point_data['create_by'] == user]

    
    #user_polys.to_file(f"/Users/ckruse/Downloads/{user.split('@')[0]}.geojson", driver='GeoJSON')

    
    overlapping = []
    for point in user_points['geometry']:
        user_polys = user_points.copy()
        user_polys['geometry'] = [geom.centroid.buffer(0.0001) for geom in user_points['geometry']]
        index = user_polys['geometry'].sindex
        num_overlaps = len(index.query(point))
        if num_overlaps > 1:
            overlapping.append(True)
            user_points.drop(user_points.index[index.query(point)[1:]], inplace=True)
            #user_polys.iloc[index.query(point)].plot()
        else:
            overlapping.append(False)
    #print(user, overlap_count)

    user_polys = user_points.copy()
    user_polys['geometry'] = [geom.centroid.buffer(0.01) for geom in user_points['geometry']]
    
    positive_class = user_polys[user_polys['label'] == 1]
    negative_class = user_polys[user_polys['label'] == 0]
    uncertain_class = user_polys[user_polys['label'] == 2]
    #print(f"{len(positive_class)} positive labels and {len(negative_class)} negative labels for {user}. {len(uncertain_class)} sites listed as unknown.")
    
    positive_count = query_df(positive_labels, positive_class)
    negative_count = query_df(negative_labels, negative_class)
    #print(f"{positive_count} true positives {positive_count / (len(positive_class)+0.01):.1%}, {negative_count} true negatives ({negative_count / (len(negative_class)+0.01):.1%})")

    results['user'].append(user.split('@')[0])
    results['total labels'].append(len(user_polys))
    results['overlapping labels'].append(sum(overlapping))
    results['positive labels'].append(len(positive_class))
    results['negative labels'].append(len(negative_class))
    results['uncertain labels'].append(len(uncertain_class))
    results['true positive count'].append(positive_count)
    results['true negative count'].append(negative_count)
    results['confirmed correct count'].append(positive_count + negative_count)
    results['positive accuracy'].append(100* np.round(positive_count / (len(positive_class)+0.01), 2))
    results['negative accuracy'].append(100* np.round(negative_count / (len(negative_class)+0.01), 2))

results_df = pd.DataFrame(results)
results_df.set_index('user', inplace=True)



/var/folders/qp/7mx_414x647gwz3z5m2rl7780000gn/T/ipykernel_81735/3267242984.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_points.drop(user_points.index[index.query(point)[1:]], inplace=True)
/var/folders/qp/7mx_414x647gwz3z5m2rl7780000gn/T/ipykernel_81735/3267242984.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_points.drop(user_points.index[index.query(point)[1:]], inplace=True)
/var/folders/qp/7mx_414x647gwz3z5m2rl7780000gn/T/ipykernel_81735/3267242984.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

In [148]:
results_df = results_df.sort_values(by='total labels', ascending=False)
results_df

,total labels,overlapping labels,positive labels,negative labels,uncertain labels,true positive count,true negative count,confirmed correct count,positive accuracy,negative accuracy
user,,,,,,,,,,
thelxinoe-5298-4e19,261,24,259,2,0,80,1,81,31.0,50.0
kale-6d35-4d93,249,247,77,136,36,40,68,108,52.0,50.0
sinope-91d0-4534,227,23,194,33,0,24,8,32,12.0,24.0
thelxinoe-fbf3-4ddc,218,10,216,2,0,28,0,28,13.0,0.0
kore-f892-4b9b,210,0,207,3,0,24,0,24,12.0,0.0
muska-6383-4d9c,187,6,13,174,0,3,0,3,23.0,0.0
pasiphae-566a-4fa4,118,14,111,7,0,4,2,6,4.0,29.0
iocaste-eeef-456a,110,1,105,5,0,70,3,73,67.0,60.0
s-2003-j9-ff60-4869,100,1,99,1,0,4,0,4,4.0,0.0


In [149]:
results_df.to_csv('/Users/ckruse/Downloads/education_challenge_results.csv')

In [151]:
overlapping = []
for point in point_data['geometry']:
    user_polys = point_data.copy()
    user_polys['geometry'] = [geom.centroid.buffer(0.0001) for geom in point_data['geometry']]
    index = user_polys['geometry'].sindex
    num_overlaps = len(index.query(point))
    if num_overlaps > 1:
        overlapping.append(True)
        point_data.drop(point_data.index[index.query(point)[1:]], inplace=True)
        #user_polys.iloc[index.query(point)].plot()
    else:
        overlapping.append(False)

In [152]:
point_data.to_file('/Users/ckruse/Downloads/deduped_education_data.geojson', driver='GeoJSON')

/Users/ckruse/miniforge3/envs/m1-plastics/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
